### Plan of Attack:

1. Get list of new songs after a specific date
2. Get data needed for those songs
3. Create parquet table
4. Save to S3

In [ ]:
# import spotipy
# import os
# %run './credentials/settings.py'
# from spotipy.oauth2 import SpotifyOAuth
# scope = "user-library-read"

# def user_auth(scope=scope):
#     SPOTIPY_CLIENT_ID = os.getenv("SPOTIPY_CLIENT_ID")
#     SPOTIPY_CLIENT_SECRET = os.getenv("SPOTIPY_CLIENT_SECRET")
#     SPOTIPY_REDIRECT_URI = os.getenv('SPOTIPY_REDIRECT_URI_READ')
#     print()
#     sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
#     return sp
# sp = user_auth()

In [1]:
!pip install spotipy

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached spotipy-2.19.0-py3-none-any.whl (27 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: six
    Found existing installation: six 1.14.0
    Uninstalling six-1.14.0:
      Successfully uninstalled six-1.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
aiobotoc

In [56]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import os
import pandas as pd
import numpy as np
%run '../../credentials/settings.py'
from collections import defaultdict

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                                           client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))

In [4]:
new_releases = sp.new_releases(country='US')

### list items we want from each album, will need to figure out how to get track ids from each album (even if an artist releases a single their is an "album" associated)
- release_date
- total_tracks
- id
- name
- artist(s)
- artist ids

In [28]:
def album_metadata(album_items):
          
    """
    The function extracts data new_releases json response from Spotipy into a pandas df
    
    """
    album_data = {}
    album_data['album_id'] = []
    album_data['release_date'] = []
    album_data['total_tracks']  = []
    album_data['artist_id_1'] = []
    album_data['artist_ids'] = []
    album_data['artist_names'] = []
    
    for album_item in album_items:
    
        # map 1:1 data for each record
        album_data['album_id'].append(album_item['id'])
        album_data['release_date'].append(album_item['release_date'])
        album_data['total_tracks'].append(album_item['total_tracks'])

        # get artist(s) data
        artist_lst = album_item['artists']
        artist_ids = [x['id'] for x in artist_lst]
        album_data['artist_id_1'].append(artist_ids[0])
        album_data['artist_ids'].append(str(artist_ids))
        album_data['artist_names'].append(str([x['name'] for x in artist_lst]))
    
    return pd.DataFrame.from_dict(album_data,orient='columns')

In [29]:
#test it
album_df = album_metadata(new_releases['albums']['items'])
album_df

,album_id,release_date,total_tracks,artist_id_1,artist_ids,artist_names
0,7jMo8dhMIorHoLlzTisjJy,2021-08-13,1,56oDRnqbIiwx4mymNEv7dS,"['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo...","['Lizzo', 'Cardi B']"
1,2TvfE8CY37OQIPVGcWYpEA,2021-08-12,1,2o5jDhtHVPhrJdv3cEQ99Z,"['2o5jDhtHVPhrJdv3cEQ99Z', '790FomKkXshlbRYZFt...","['Tiësto', 'KAROL G']"
2,5D8Rdb09BkmHscEGSWAlA6,2021-08-13,1,3PhoLpVuITZKcymswpck5b,"['3PhoLpVuITZKcymswpck5b', '6M2wZ9GZgrQXHCFfjv...","['Elton John', 'Dua Lipa', 'PNAU']"
3,00fcDjvEq4elj756TESO0c,2021-08-13,1,78rUTD7y6Cy67W1RVzYs7t,['78rUTD7y6Cy67W1RVzYs7t'],['PinkPantheress']
4,1qMFjBarjO2xD15BwXZguD,2021-08-13,11,0C0XlULifJtAgn6ZNCW2eu,['0C0XlULifJtAgn6ZNCW2eu'],['The Killers']
5,6vqNFOfIgqe0l2Ui47YSyO,2021-08-13,6,2cFrymmkijnjDg9SS92EPM,['2cFrymmkijnjDg9SS92EPM'],['blackbear']
6,4ih8LPpqdYPezAMaPAHJYT,2021-08-07,1,46SHBwWsqBkxI7EeeBEQG7,"['46SHBwWsqBkxI7EeeBEQG7', '45TgXXqMDdF8BkjA83...","['Kodak Black', 'Rod Wave']"
7,3k4lUXBZxjUGFFQvy8PsHn,2021-08-12,1,6TIYQ3jFPwQSRmorSezPxX,['6TIYQ3jFPwQSRmorSezPxX'],['Machine Gun Kelly']
8,4pWo1MFBvVAfhjL4aPZFaD,2021-08-13,1,1vyhD5VmyZ7KMfW5gqLgo5,['1vyhD5VmyZ7KMfW5gqLgo5'],['J Balvin']
9,7L8IHgiomfEpxOm61vgPTm,2021-08-13,12,7z5WFjZAIYejWy0NI5lv4T,['7z5WFjZAIYejWy0NI5lv4T'],['Dan + Shay']


### figure out how to get track IDs from each of album avoe of the above

In [50]:
def tracks_from_album(df):
      
    """
    The function uses Spotipy to fetch track data for all albums released in album_df.
    
    """
    data = {}
    data['album_id'] = []
    data['track_id'] = []
    data['track_name'] = []
    data['artist_id_1'] = []
    data['artist_id_2'] = []
    data['artist_ids'] = []
    data['artist_names'] = []
    data['explicit'] = []
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                                           client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))
    for i,x in df.iterrows():
        album_id = x['album_id']
        track_data = sp.album_tracks(album_id)
        items = track_data['items']

        for item in items:

            # map 1:1 data for each record
            data['album_id'].append(album_id)
            data['track_id'].append(item['id'])
            data['track_name'].append(item['name'])
            data['explicit'].append(item['explicit'])

            # get artist(s) data
            artist_lst = item['artists']
            artist_ids = [x['id'] for x in artist_lst]
            data['artist_id_1'].append(artist_ids[0])
            if len(artist_ids) > 1:
                data['artist_id_2'].append(artist_ids[1])
            else:
                data['artist_id_2'].append('')
            data['artist_ids'].append(str(artist_ids))
            data['artist_names'].append(str([x['name'] for x in artist_lst]))
    
    return pd.DataFrame.from_dict(data,orient='columns')

In [51]:
%%time
tracks_df = tracks_from_album(album_df)
tracks_df.head()

CPU times: user 76.7 ms, sys: 5.85 ms, total: 82.5 ms
Wall time: 1.56 s


,album_id,track_id,track_name,artist_id_1,artist_id_2,artist_ids,artist_names,explicit
0,7jMo8dhMIorHoLlzTisjJy,6KgtcmCF9Ky68XC7ezxl3s,Rumors (feat. Cardi B),56oDRnqbIiwx4mymNEv7dS,4kYSro6naA4h99UJvo89HB,"['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo...","['Lizzo', 'Cardi B']",True
1,2TvfE8CY37OQIPVGcWYpEA,0bI7K9Becu2dtXK1Q3cZNB,Don't Be Shy,2o5jDhtHVPhrJdv3cEQ99Z,790FomKkXshlbRYZFtlgla,"['2o5jDhtHVPhrJdv3cEQ99Z', '790FomKkXshlbRYZFt...","['Tiësto', 'KAROL G']",False
2,5D8Rdb09BkmHscEGSWAlA6,6zSpb8dQRaw0M1dK8PBwQz,Cold Heart - PNAU Remix,3PhoLpVuITZKcymswpck5b,6M2wZ9GZgrQXHCFfjv46we,"['3PhoLpVuITZKcymswpck5b', '6M2wZ9GZgrQXHCFfjv...","['Elton John', 'Dua Lipa', 'PNAU']",False
3,00fcDjvEq4elj756TESO0c,6OTKVgVpVaVjhRLYizPJKA,Just for me,78rUTD7y6Cy67W1RVzYs7t,,['78rUTD7y6Cy67W1RVzYs7t'],['PinkPantheress'],False
4,1qMFjBarjO2xD15BwXZguD,5hiZJE6Fg14Wao6EJ0KUC8,West Hills,0C0XlULifJtAgn6ZNCW2eu,,['0C0XlULifJtAgn6ZNCW2eu'],['The Killers'],False


### merge to get release date of album

In [52]:
df = tracks_df.merge(album_df[['album_id', 'release_date']],how='left',on='album_id')
df.head()

,album_id,track_id,track_name,artist_id_1,artist_id_2,artist_ids,artist_names,explicit,release_date
0,7jMo8dhMIorHoLlzTisjJy,6KgtcmCF9Ky68XC7ezxl3s,Rumors (feat. Cardi B),56oDRnqbIiwx4mymNEv7dS,4kYSro6naA4h99UJvo89HB,"['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo...","['Lizzo', 'Cardi B']",True,2021-08-13
1,2TvfE8CY37OQIPVGcWYpEA,0bI7K9Becu2dtXK1Q3cZNB,Don't Be Shy,2o5jDhtHVPhrJdv3cEQ99Z,790FomKkXshlbRYZFtlgla,"['2o5jDhtHVPhrJdv3cEQ99Z', '790FomKkXshlbRYZFt...","['Tiësto', 'KAROL G']",False,2021-08-12
2,5D8Rdb09BkmHscEGSWAlA6,6zSpb8dQRaw0M1dK8PBwQz,Cold Heart - PNAU Remix,3PhoLpVuITZKcymswpck5b,6M2wZ9GZgrQXHCFfjv46we,"['3PhoLpVuITZKcymswpck5b', '6M2wZ9GZgrQXHCFfjv...","['Elton John', 'Dua Lipa', 'PNAU']",False,2021-08-13
3,00fcDjvEq4elj756TESO0c,6OTKVgVpVaVjhRLYizPJKA,Just for me,78rUTD7y6Cy67W1RVzYs7t,,['78rUTD7y6Cy67W1RVzYs7t'],['PinkPantheress'],False,2021-08-13
4,1qMFjBarjO2xD15BwXZguD,5hiZJE6Fg14Wao6EJ0KUC8,West Hills,0C0XlULifJtAgn6ZNCW2eu,,['0C0XlULifJtAgn6ZNCW2eu'],['The Killers'],False,2021-08-13


### get audio features for every track

In [61]:
track_ids = df.track_id.tolist()
audio_features = sp.audio_features(track_ids)

In [62]:
len(audio_features)

83

In [53]:
audio_features[0]

{'danceability': 0.827,
 'energy': 0.731,
 'key': 4,
 'loudness': -5.524,
 'mode': 0,
 'speechiness': 0.088,
 'acousticness': 0.13,
 'instrumentalness': 0,
 'liveness': 0.418,
 'valence': 0.607,
 'tempo': 118.992,
 'type': 'audio_features',
 'id': '6KgtcmCF9Ky68XC7ezxl3s',
 'uri': 'spotify:track:6KgtcmCF9Ky68XC7ezxl3s',
 'track_href': 'https://api.spotify.com/v1/tracks/6KgtcmCF9Ky68XC7ezxl3s',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6KgtcmCF9Ky68XC7ezxl3s',
 'duration_ms': 172833,
 'time_signature': 4}

In [63]:
for key, value in audio_features[0].items():
    print(key,value)

danceability 0.827
energy 0.731
key 4
loudness -5.524
mode 0
speechiness 0.088
acousticness 0.13
instrumentalness 0
liveness 0.418
valence 0.607
tempo 118.992
type audio_features
id 6KgtcmCF9Ky68XC7ezxl3s
uri spotify:track:6KgtcmCF9Ky68XC7ezxl3s
track_href https://api.spotify.com/v1/tracks/6KgtcmCF9Ky68XC7ezxl3s
analysis_url https://api.spotify.com/v1/audio-analysis/6KgtcmCF9Ky68XC7ezxl3s
duration_ms 172833
time_signature 4


In [69]:
song = audio_features[0]

In [71]:
type(song['danceability'])

float

In [43]:
df.shape

(83, 8)

In [79]:
def audio_features_from_tracks(track_ids):
  
    """
    The function uses Spotipy to fetch audio features and metadata for the specified track_ids.
    
    """
    
    song_data = defaultdict()
    audio_features = sp.audio_features(track_ids)
    
    for i,song in enumerate(audio_features):
        if i == 0:
            for key, value in song.items():
                song_data[key] = [value]
        else:
            for key, value in song.items():
                song_data[key].append(value)
    
    df = pd.DataFrame(song_data)
    df = df.rename(columns={'id':'track_id'})
    df = df.drop(columns=['track_href','analysis_url','time_signature','uri','type'])
                
    return df
    

In [80]:
%%time
track_ids = df.track_id.tolist()
audio_df = audio_features_from_tracks(track_ids)
audio_df.head()

CPU times: user 11.1 ms, sys: 2.99 ms, total: 14.1 ms
Wall time: 159 ms


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_id,duration_ms
0,0.827,0.731,4,-5.524,0,0.0880,0.130,0.000000,0.4180,0.607,118.992,6KgtcmCF9Ky68XC7ezxl3s,172833
1,0.770,0.787,11,-3.748,0,0.0583,0.198,0.000000,0.2620,0.513,119.986,0bI7K9Becu2dtXK1Q3cZNB,140500
2,0.796,0.798,1,-6.312,1,0.0317,0.034,0.000042,0.0952,0.942,116.032,6zSpb8dQRaw0M1dK8PBwQz,202735
3,0.778,0.628,0,-7.844,1,0.0618,0.705,0.001930,0.2990,0.698,133.048,6OTKVgVpVaVjhRLYizPJKA,115801
4,0.318,0.670,0,-6.306,0,0.0528,0.475,0.003950,0.5770,0.190,169.893,5hiZJE6Fg14Wao6EJ0KUC8,342013


### get genres for every artist